<a href="https://colab.research.google.com/github/YuvanBharathi13/DeepLearning/blob/main/05_transfer_learning_in_tensorflow_part_2_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

zip_ref = zipfile.ZipFile('10_food_classes_10_percent.zip')
zip_ref.extractall()
zip_ref.close()

--2025-06-15 04:55:57--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.207, 172.253.117.207, 142.250.99.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   237MB/s    in 0.7s    

2025-06-15 04:55:58 (237 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



#Pre processing data

In [9]:
import os
def walk_through_dir(dir_path):
  for dirpath, dir_name, files in os.walk(dir_path):
    print(f"There are {len(dir_name)} folders, {len(files)} files in {dirpath}")

In [11]:
walk_through_dir('10_food_classes_10_percent')

There are 2 folders, 0 files in 10_food_classes_10_percent
There are 10 folders, 0 files in 10_food_classes_10_percent/test
There are 0 folders, 250 files in 10_food_classes_10_percent/test/ice_cream
There are 0 folders, 250 files in 10_food_classes_10_percent/test/chicken_curry
There are 0 folders, 250 files in 10_food_classes_10_percent/test/fried_rice
There are 0 folders, 250 files in 10_food_classes_10_percent/test/steak
There are 0 folders, 250 files in 10_food_classes_10_percent/test/hamburger
There are 0 folders, 250 files in 10_food_classes_10_percent/test/sushi
There are 0 folders, 250 files in 10_food_classes_10_percent/test/chicken_wings
There are 0 folders, 250 files in 10_food_classes_10_percent/test/ramen
There are 0 folders, 250 files in 10_food_classes_10_percent/test/grilled_salmon
There are 0 folders, 250 files in 10_food_classes_10_percent/test/pizza
There are 10 folders, 0 files in 10_food_classes_10_percent/train
There are 0 folders, 75 files in 10_food_classes_10_

In [7]:
import pathlib
def get_class_names(train_path):
  data_dir = pathlib.Path(train_path)
  class_names = np.array(sorted(item.name for item in data_dir.glob('*')))
  return class_names

In [8]:
get_class_names('10_food_classes_10_percent/test')

array(['chicken_curry', 'chicken_wings', 'fried_rice', 'grilled_salmon',
       'hamburger', 'ice_cream', 'pizza', 'ramen', 'steak', 'sushi'],
      dtype='<U14')

In [12]:
train_dir = '10_food_classes_10_percent/train'
test_dir = '10_food_classes_10_percent/test'

- instead of using ImageDataGenerator, this time we will be using tf.keras.preprocessing.image_dataset_from_directory(), which returns a tf.data.Dataset object.
- This type of dataset is faster than ImageDataGeberator API and comes with multiple functions.

In [18]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 image_size=(224,224),
                                                                 batch_size=32,
                                                                 label_mode='categorical')
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode='categorical',
                                                                image_size=(224,224),
                                                                batch_size=32)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [19]:
train_data.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

- below we can see that the values of images are not scaled and the labels are all ONE HOT ENCODED.
- there is no need to scale the images now as most models have a scale layer within them that does the job.(resnet doesnt have)

In [20]:
for images, labels in train_data.take(1):
  print(images)
  print(labels)

tf.Tensor(
[[[[5.10000000e+01 6.10000000e+01 6.30000000e+01]
   [5.10255089e+01 6.10255089e+01 6.30255089e+01]
   [4.75663261e+01 5.75663261e+01 5.93520393e+01]
   ...
   [7.29948196e+01 3.87805557e+01 1.31377621e+01]
   [7.13061371e+01 3.72346954e+01 1.08265114e+01]
   [7.50153656e+01 3.73010063e+01 7.58665228e+00]]

  [[4.58571396e+01 5.58571396e+01 5.78571396e+01]
   [4.81428566e+01 5.81428566e+01 6.01428566e+01]
   [4.61428566e+01 5.61428566e+01 5.79285698e+01]
   ...
   [7.27194672e+01 3.89031219e+01 1.10919819e+01]
   [8.04438934e+01 4.63724556e+01 1.82958965e+01]
   [7.99336777e+01 4.28621483e+01 1.05049706e+01]]

  [[4.17857132e+01 5.13571434e+01 5.33571434e+01]
   [4.18571396e+01 5.14285698e+01 5.34285698e+01]
   [3.79540825e+01 4.75255089e+01 4.87397957e+01]
   ...
   [6.70713425e+01 3.56428146e+01 6.21428585e+00]
   [6.96020737e+01 3.64591904e+01 5.38774776e+00]
   [7.32908936e+01 3.65765381e+01 1.79078960e+00]]

  ...

  [[9.42756348e+01 3.51328201e+01 1.15716248e+01]
   [1

# Creating models

## Model 0

In [21]:
# create the base model(trained) from tf.applications
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False)

# freeze the weights
base_model.trainable = False

# create the input layer with input shape as (224,224,3)
inputs = tf.keras.layers.Input(shape=(224,224,3),name='input_layer')

# suppose you are using resnet which does not have a scaling layer we must scale it
# x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)

# pass the output of the input layer to the base_model
x = base_model(inputs)

# pass the ouput pf the base_model to the average pooling layer which
# aggregates most important information and reduces number of computations
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)

# pass the output of the average layer to the output layer
outputs = tf.keras.layers.Dense(10, activation="softmax", name="output_layer")(x)

# create the model using the inputs and outputs layers
model_0 = tf.keras.Model(inputs,outputs)

24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [22]:
model_0.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [23]:
zero_history = model_0.fit(train_data,
                           epochs=5,
                           steps_per_epoch=len(train_data),
                           validation_data=test_data,
                           validation_steps=len(test_data))

Epoch 1/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2933 - loss: 2.1020

KeyboardInterrupt: 